In [ ]:
import pandas as pd

file_name = '/content/diabetes.csv'
df = pd.read_csv(file_name)

In [ ]:
import numpy as np
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state= 24)

In [ ]:
# Feature Scalling

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Create Custom Dataset

class Customdataset(Dataset):
    def __init__(self,X,y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_dataset = Customdataset(X_train, y_train)
test_dataset = Customdataset(X_train, X_test)

train_dataloader = DataLoader(Customdataset(X_train, y_train) , batch_size=32, shuffle=True)
test_dataloader = DataLoader(Customdataset(X_test, y_test) , batch_size=32, shuffle=False)

In [ ]:
# Create NN

import torch.nn as nn

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(8, 28),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(28, 14),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(14, 2)
        )

    def forward(self, X):
        return self.model(X)

    def forward(self, X):
        return self.model(X)

In [ ]:
# Adjust HyperParameter

epochs = 500
learning_rate= 0.01
model = NN()
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
# Training Loop

model.train

for epoch in range(epochs):
    for X_batch, y_batch in train_dataloader:

        # Convert to float
        X_batch = X_batch.float()
        y_batch = y_batch.long()

        # Forward Pass
        outputs = model(X_batch)

        # Calculate Loss
        loss_value = loss(outputs, y_batch)

        # BackProp and Update Param
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix

def evaluate_model(model, dataloader, task_type='classification', device='cpu'):

    model.to(device)
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)

            if task_type == 'classification':
                _, predicted = torch.max(outputs, 1)  # for multi-class classification
                y_pred.extend(predicted.cpu().numpy())
                y_true.extend(targets.cpu().numpy())

            elif task_type == 'regression':
                y_pred.extend(outputs.squeeze().cpu().numpy())
                y_true.extend(targets.squeeze().cpu().numpy())

    metrics = {}

    if task_type == 'classification':
        metrics['accuracy'] = accuracy_score(y_true, y_pred)
        metrics['report'] = classification_report(y_true, y_pred)
        metrics['confusion_matrix'] = confusion_matrix(y_true, y_pred)
    elif task_type == 'regression':
        metrics['mse'] = mean_squared_error(y_true, y_pred)
        metrics['rmse'] = mean_squared_error(y_true, y_pred, squared=False)

    return metrics

In [ ]:
# Example usage
test_dataloader = DataLoader(Customdataset(X_test, y_test) , batch_size=32, shuffle=False)
metrics = evaluate_model(model, test_dataloader, task_type='classification', device='cpu')

print(metrics)


{'accuracy': 0.7077922077922078, 'report': '              precision    recall  f1-score   support\n\n         0.0       0.70      0.96      0.81        98\n         1.0       0.79      0.27      0.40        56\n\n    accuracy                           0.71       154\n   macro avg       0.74      0.61      0.60       154\nweighted avg       0.73      0.71      0.66       154\n', 'confusion_matrix': array([[94,  4],
       [41, 15]])}
